SQLite Database Initialization
This section initializes the SQLite database by creating necessary tables such as Items, Auctions, and ActionEvents. It ensures the existence of the required tables and sets up the database schema.

File and Configuration Setup
Additionally, it creates other essential files and configurations needed for subsequent operations.

In [2]:
!rm auction.db

"rm" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [3]:
import sqlite3
import os

db_path = 'auction.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Items (
        item_id INT PRIMARY KEY,
        item_name TEXT,
        quality TEXT,
        item_level INT,
        required_level INT,
        item_class TEXT,
        item_subclass TEXT,
        purchase_price_gold INT,
        purchase_price_silver INT,
        sell_price_gold INT,
        sell_price_silver INT,
        max_count INT,
        is_stackable INT
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Auctions (
        auction_id INT PRIMARY KEY,
        bid INT,
        buyout INT,
        quantity INT,
        time_left TEXT,
        item_id INT
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS ActionEvents (
        auction_id INT,
        record DATETIME,
        PRIMARY KEY (auction_id, record),
        FOREIGN KEY (auction_id) REFERENCES Auctions(auction_id)
    )
''')

conn.commit()
conn.close()

In [4]:
from datetime import datetime
from tqdm import tqdm

file_info = {}
data_dir = 'sample/'

for root, dirs, files in os.walk(data_dir):
    for filename in tqdm(files):
        filepath = os.path.join(root, filename)
        date = datetime.strptime(filename.split('.')[0], '%Y%m%dT%H')

        file_info[filepath] = date

file_info = {k: v for k, v in sorted(file_info.items(), key=lambda item: item[1])}
filenames = list(file_info.keys())

MySQL Items Data Retrieval
In this part, the script retrieves data from a MySQL database. It reads the MySQL database configuration from a JSON file, establishes a connection, and fetches data from the Items table. The retrieved data is then loaded into a Pandas DataFrame.

SQLite Database Update
After retrieving the data, the script connects to the SQLite database, deletes all existing records from the Items table, and appends the newly fetched data.

In [5]:
import json
import os
from datetime import datetime
from tqdm import tqdm
import sqlite3

db_path = 'auction.db'
data_dir = 'sample/'
db = sqlite3.connect(db_path)
cursor = db.cursor()

for i, filepath in tqdm(enumerate(filenames)):
    try:
        data = json.load(open(filepath, "r"))
    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"Error reading file {filepath}: {e}")
        continue

    auction_record = datetime.strptime(filename[:-5], "%Y%m%dT%H")

    if i == 0:
        auction_ids = []
        auctions_data = []

        for auction in data["auctions"]:
            if auction["id"] not in auction_ids:
                auctions_data.append((auction["id"], auction["bid"], auction["buyout"], auction["quantity"], auction["time_left"], auction["item"]["id"]))
                auction_ids.append(auction["id"])

        try:
            cursor.executemany("""
                INSERT INTO Auctions (auction_id, bid, buyout, quantity, time_left, item_id)
                VALUES (?, ?, ?, ?, ?, ?)
            """, auctions_data)
            db.commit()
        except sqlite3.Error as err:
            db.rollback()
            print(f"Error inserting auction data for file {filepath} in Auctions: {err}")

    action_events_data = []
    for auction in data["auctions"]:
        action_events_data.append((auction["id"], auction_record.strftime('%Y-%m-%d %H:%M:%S')))
            
    try:
        cursor.executemany("""
            INSERT OR REPLACE INTO ActionEvents (auction_id, record)
            VALUES (?, ?)
        """, action_events_data)
        db.commit()
    except sqlite3.Error as err:
        db.rollback()
        print(f"Error inserting auction events for file {filepath} in ActionEvents: {err}")

cursor.close()
db.close()

0it [00:00, ?it/s]


Auction Data and Items Storage
This section processes JSON files containing auction data. It iterates through the files, extracts relevant information, and inserts it into the Auctions and ActionEvents tables of the SQLite database.

Data Import from Pandas DataFrame
Moreover, it initializes a connection to the SQLite database and imports data from a Pandas DataFrame into the Items table. This DataFrame is generated from MySQL database retrieval, ensuring the SQLite Items table is up-to-date.

In [6]:
import pandas as pd
import mysql.connector
import json
import sqlite3

with open('../data/config.json') as f:
    config = json.load(f)

db_path = 'auction.db'
query = "SELECT * FROM Items"

def import_items():
    try:
        mysql_db = mysql.connector.connect(**config['database'])
    except mysql.connector.Error as err:
        print(err)
        return

    cursor = mysql_db.cursor()

    items = cursor.execute(query)
    items = cursor.fetchall()

    mysql_db.close()
    cursor.close()

    try:
        db = sqlite3.connect(db_path)
        print("Connected to SQLite")
    except sqlite3.Error as err:
        return

    cursor = db.cursor()
    cursor.executemany("""
        INSERT OR REPLACE INTO Items (item_id, item_name, quality, item_level, required_level, item_class, item_subclass, purchase_price_gold, purchase_price_silver, sell_price_gold, sell_price_silver, max_count, is_stackable)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, items)

    db.commit()

    cursor.close()
    db.close()

    print("Inserted items into SQLite: " + str(len(items)))

import_items()

Connected to SQLite
Inserted items into SQLite: 10396


It is responsible for connecting to an SQLite database, executing a SQL query involving multiple tables, and retrieving the results. These results are stored in the variable

In [1]:
import sqlite3
import pandas as pd
import sys

# Asegúrate de que la ruta sea correcta para tu archivo transformers.py
sys.path.insert(0, 'C:\\Users\\CMIRANDA\\Documents\\auction_box\\data')

from transformers import TimeLeftTransformer, MedianCompetitorPriceTransformer, AvgCompetitorPriceTransformer, CompetitorCountTransformer, MinCompetitorPriceTransformer, TopCompetitorPriceTransformer , RelativeDifferencesTransformer

db_path = 'auction.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

query = """
    SELECT
        a.auction_id,
        a.bid / 10000 AS bid_in_gold,
        a.buyout / 10000 AS buyout_in_gold,
        (a.buyout / 10000) / a.quantity AS unit_price,
        a.quantity,
        a.time_left,
        a.item_id,
        i.item_name,
        i.quality,
        i.item_class,
        i.item_subclass,
        i.is_stackable,
        i.purchase_price_gold,
        i.required_level,
        i.item_level,
        i.sell_price_gold,
        MIN(ae.record) AS first_appearance_timestamp,
        strftime('%Y', MIN(ae.record)) AS first_appearance_year,
        strftime('%m', MIN(ae.record)) AS first_appearance_month,
        strftime('%d', MIN(ae.record)) AS first_appearance_day,
        strftime('%H', MIN(ae.record)) AS first_appearance_hour,
        COUNT(*) AS hours_on_sale
    FROM Auctions a
    JOIN ActionEvents ae ON a.auction_id = ae.auction_id
    JOIN Items i ON i.item_id = a.item_id
    GROUP BY a.auction_id
"""

cursor.execute(query)
results = cursor.fetchall()

conn.close()

df = pd.DataFrame(results, columns=[i[0] for i in cursor.description])


time_left_transformer = TimeLeftTransformer()
df = time_left_transformer.fit_transform(df)


median_competitor_price_transformer = MedianCompetitorPriceTransformer()
df = median_competitor_price_transformer.fit_transform(df)


avg_competitor_price_transformer = AvgCompetitorPriceTransformer()
df = avg_competitor_price_transformer.fit_transform(df)


competitor_count_transformer = CompetitorCountTransformer()
df = competitor_count_transformer.fit_transform(df)


min_competitor_price_transformer = MinCompetitorPriceTransformer()
df = min_competitor_price_transformer.fit_transform(df)


top_competitor_price_transformer = TopCompetitorPriceTransformer()
df = top_competitor_price_transformer.fit_transform(df)

relative_differences_transformer = RelativeDifferencesTransformer()
df = relative_differences_transformer.fit_transform(df)


df.head()



,auction_id,bid_in_gold,buyout_in_gold,unit_price,quantity,time_left,item_id,item_name,quality,item_class,...,std_competitor_price,competitor_count,lowest_competitor_price,top_competitor_price,relative_price_difference,relative_buyout_difference,relative_bid_difference,relative_price_to_lowest_competitor,relative_price_to_top_competitor,relative_avg_price_difference
0,969819641,0,0,0,1,0.5,8398,Pattern: Tough Scorpid Gloves,Uncommon,Recipe,...,0.000000,17,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
1,969819643,0,0,0,1,0.5,8398,Pattern: Tough Scorpid Gloves,Uncommon,Recipe,...,0.000000,17,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
2,969819653,0,0,0,1,0.5,8395,Pattern: Tough Scorpid Breastplate,Uncommon,Recipe,...,2.529822,10,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,-0.316228
3,969819911,27,28,1,20,0.5,21840,Bolt of Netherweave,Common,Trade Goods,...,460.412447,382,1.0,1.0,0.0,3.0,3.499999,0.0,0.0,-0.051900
4,969819913,27,28,1,20,0.5,21840,Bolt of Netherweave,Common,Trade Goods,...,460.412447,382,1.0,1.0,0.0,3.0,3.499999,0.0,0.0,-0.051900


organize the results of a SQL query into a DataFrame, and then apply some form of preprocessing to that data using functions from a module called preprocess_data.

In [3]:
numerical_columns = [
    'quantity',
    'unit_price',
    'bid_in_gold',
    'buyout_in_gold',
    'time_left',
    'median_buyout_price',
    'median_bid_price',
    'median_unit_price',
    'lowest_competitor_price',
    'avg_competitor_price',
    'std_competitor_price',
    'top_competitor_price',
    'competitor_count',
    'rank_buyout_price',
    'rank_bid_price',
    'rank_unit_price',
    'relative_price_difference',
    'relative_avg_price_difference',
    'relative_buyout_difference',
    'relative_bid_difference',
    'relative_price_to_lowest_competitor',
    'relative_price_to_top_competitor',
    'purchase_price_gold',
    'sell_price_gold',
    'required_level',
    'item_level'
]

categorical_columns_ordinal = [
    'item_id',
    'quality',
    'item_class',
    'item_subclass'
]
categorical_columns_onehot = [
  'is_stackable'
]

X = df[numerical_columns + categorical_columns_ordinal + categorical_columns_onehot]
y = df['hours_on_sale']

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer

num_transformer = StandardScaler()
ordinal_transformer = OrdinalEncoder()
onehot_transformer = OneHotEncoder(sparse_output=False)

column_transformer = make_column_transformer(
    (num_transformer, numerical_columns),
    (ordinal_transformer, categorical_columns_ordinal),
    (onehot_transformer, categorical_columns_onehot),
    remainder='passthrough'
)

preprocessing_pipeline = Pipeline(steps=[
    ('time_left', TimeLeftTransformer()),
    ('median_competitor_price', MedianCompetitorPriceTransformer()),
    ('avg_competitor_price', AvgCompetitorPriceTransformer()),
    ('competitor_count', CompetitorCountTransformer()),
    ('min_competitor_price', MinCompetitorPriceTransformer()),
    ('top_competitor_price', TopCompetitorPriceTransformer()),
    ('relative_differences', RelativeDifferencesTransformer()),
    ('column_transformer', column_transformer)
])

X = preprocessing_pipeline.fit_transform(df)
y = np.array(y)


KeyError: 'avg_competitor_price'

loads a trained model from a file, makes predictions on a data set, calculates the RMSE and displays the result

In [ ]:
rmse = mean_squared_error(y, predictions, squared=False)

print("RMSE:", rmse)

RMSE: 2129798.6409702906
